In [13]:
import gymnasium as gym
import numpy as np


class ElectricityDispatchEnv(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Box(
            low=0, high=1, shape=(3,)
        )  # Grid, battery, generator actions
        self.observation_space = gym.spaces.Box(
            low=0, high=np.inf, shape=(4,)
        )  # Demand, solar forecast, battery state, cost

        self.max_steps = 96  # 15-minute intervals for 24 hours
        self.step_count = 0

        self.demand = None
        self.solar_forecast = None
        self.battery_state = 0
        self.cost = 0

    def reset(self):
        self.step_count = 0
        self.battery_state = 0
        self.cost = 0

        self.demand = self._generate_demand()
        self.solar_forecast = self._generate_solar_forecast()

        return np.array(
            [self.demand[0], self.solar_forecast[0], self.battery_state, self.cost]
        )

    def step(self, action):
        grid_action, battery_action, generator_action = action

        grid_supply = grid_action * self.demand[self.step_count]
        battery_supply = battery_action * self.battery_state
        generator_supply = generator_action * (
            self.demand[self.step_count] - self.solar_forecast[self.step_count]
        )

        total_supply = grid_supply + battery_supply + generator_supply

        reward = -self.cost  # Negative cost as reward

        if grid_supply > 0.1 * self.demand[self.step_count]:
            reward -= 100  # Penalty for exceeding 10% grid usage

        self.battery_state = max(0, self.battery_state - battery_supply)
        self.cost += grid_supply * 0.1 + generator_supply * 0.2  # Example costs

        self.step_count += 1
        done = self.step_count >= self.max_steps

        next_state = np.array(
            [
                self.demand[self.step_count] if not done else 0,
                self.solar_forecast[self.step_count] if not done else 0,
                self.battery_state,
                self.cost,
            ]
        )

        return next_state, reward, done, {}

    def _generate_demand(self):
        # Generate random demand profile
        return np.random.normal(loc=50, scale=10, size=self.max_steps)

    def _generate_solar_forecast(self):
        # Generate random solar forecast profile
        return np.random.normal(loc=30, scale=5, size=self.max_steps)


# Create the environment
env = ElectricityDispatchEnv()

# Define the agents (example using random agents)
grid_agent = lambda obs: np.random.uniform(low=0, high=1)
battery_agent = lambda obs: np.random.uniform(low=0, high=1)
generator_agent = lambda obs: np.random.randint(0, 2)

# Run the simulation
num_episodes = 10

for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        grid_action = grid_agent(state)
        battery_action = battery_agent(state)
        generator_action = generator_agent(state)

        action = np.array([grid_action, battery_action, generator_action])

        next_state, reward, done, _ = env.step(action)

        state = next_state

    print(f"Episode {episode+1}: Cost = {-reward:.2f}")

Episode 1: Cost = 508.24
Episode 2: Cost = 589.86
Episode 3: Cost = 502.66
Episode 4: Cost = 537.75
Episode 5: Cost = 478.37
Episode 6: Cost = 498.88
Episode 7: Cost = 529.51
Episode 8: Cost = 551.11
Episode 9: Cost = 426.24
Episode 10: Cost = 440.18


In [14]:
import numpy as np


def generate_demand_profile(num_steps, base_demand, demand_std, peak_hours):
    demand_profile = np.random.normal(loc=base_demand, scale=demand_std, size=num_steps)

    # Add peak demand during specified hours
    for peak_hour in peak_hours:
        demand_profile[peak_hour : peak_hour + 4] += base_demand * 0.5

    return demand_profile


def generate_solar_forecast_profile(num_steps, base_solar, solar_std, daylight_hours):
    solar_forecast_profile = np.zeros(num_steps)

    # Generate solar forecast during daylight hours
    solar_forecast_profile[daylight_hours[0] : daylight_hours[1]] = np.random.normal(
        loc=base_solar, scale=solar_std, size=daylight_hours[1] - daylight_hours[0]
    )

    return solar_forecast_profile


# Parameters
num_steps = 96  # Number of 15-minute intervals in a day
base_demand = 50  # Base demand in MW
demand_std = 10  # Standard deviation of demand
peak_hours = [36, 48, 60]  # Peak demand hours (e.g., 9am, 12pm, 3pm)

base_solar = 30  # Base solar generation in MW
solar_std = 5  # Standard deviation of solar generation
daylight_hours = [24, 72]  # Daylight hours (e.g., 6am to 6pm)

# Generate demand profile
demand_profile = generate_demand_profile(num_steps, base_demand, demand_std, peak_hours)

# Generate solar forecast profile
solar_forecast_profile = generate_solar_forecast_profile(
    num_steps, base_solar, solar_std, daylight_hours
)

# Print the generated profiles
print("Demand Profile:")
print(demand_profile)
print("\nSolar Forecast Profile:")
print(solar_forecast_profile)

Demand Profile:
[45.22171502 43.33853969 66.07182592 56.34036181 35.17408856 57.59789962
 60.14204479 63.89303616 59.50044259 52.06903689 46.97717573 49.71000545
 36.34288168 66.00265861 30.40372701 56.64687197 41.02999917 67.20447345
 42.09800352 42.98739313 58.58698347 54.56262588 60.07048161 44.70956605
 66.98894248 62.59532029 54.14978385 56.18954253 42.9446175  64.09328933
 52.71616158 35.71506403 47.60844618 86.60222683 63.28516343 38.33037042
 59.97231771 85.21449926 70.30672358 74.41800177 42.2947841  53.11524371
 59.98488931 35.19682876 50.321872   58.7905803  53.84413348 44.09291996
 64.71315448 97.61308722 65.95078945 73.61142223 54.96326531 44.59439054
 40.54746954 67.24643919 47.49399604 59.38681214 58.53701528 59.30818679
 72.03696026 66.22884168 71.03549794 89.39315832 62.71131191 32.79893398
 32.17588969 58.33680034 44.52898999 58.47634845 38.54268669 56.42743717
 61.90424327 54.51633203 46.04237686 45.14124996 48.44211776 53.95207462
 60.88105095 51.64830682 60.0155209